In [ ]:
import numpy as np
import scipy as sc
from matplotlib import pyplot as plt


In [ ]:
delta_time: float = 0.035

simulator_params: dict = {
    "dt": delta_time,
    "r": 0.10,
    "m": 3,
    "Jv": 5,
    "kt": 10e2,
    "kb": 10e-3,
    "ra": 10e-1,
    "ngb": 7,
    "psi": np.deg2rad([100, 100, 100]).transpose(),
    "L": np.array([0.2, 0.2, 0.2]).transpose(),
    "delta": np.deg2rad([60, 180, 60]).transpose(),
}
controller_params: dict = {
    "dt": delta_time,
}


class Simulator:
    def __init__(self, params: dict):
        self.params = params
        self.params["alpha"] = self.params["delta"] - self.param["psi"]

        self.MC = np.zeros(shape=(3, 3))
        self.MC[0, 0] = -np.cos(self.params["alpha"][0])
        self.MC[0, 1] = -np.sin(self.params["alpha"][0])
        self.MC[0, 2] = self.params["L"][0] * np.sin(self.params["psi"][0])
        self.MC[1, 0] = -np.cos(self.params["alpha"][1])
        self.MC[1, 1] = -np.sin(self.params["alpha"][1])
        self.MC[1, 2] = self.params["L"][1] * np.sin(self.params["psi"][1])
        self.MC[2, 0] = -np.cos(self.params["alpha"][2])
        self.MC[2, 1] = -np.sin(self.params["alpha"][2])
        self.MC[2, 2] = self.params["L"][2] * np.sin(self.params["psi"][2])

        self.MM = np.diag(
            [1 / self.params["m"], 1 / self.params["m"], 1 / self.params["Jv"]]
        )

        self.MA = (
            np.eye(3)
            * (1 / self.params["r"])
            * (self.params["kt"] * self.params["ngb"] / self.params["ra"])
        )
        self.MB = (
            np.eye(3)
            * (1 / self.params["r"])
            * (
                self.params["kt"]
                * self.params["kb"]
                * self.params["ngb"]
                * self.params["ngb"]
                / self.params["ra"]
            )
        )
        self.M1 = (1 / self.params["r"]) * self.MC @ self.MM @ self.MC.T @ self.MA
        self.M2 = (1 / self.params["r"]) * self.MC @ self.MM @ self.MC.T @ self.MB

    def dynamical_model(self,x,t,v):
        omega_dot = np.dot(self.M1,v) - np.dot(self.M2,x)
        return omega_dot

class Controller:
    def __init__(self, params: dict):
        self.params = params

In [ ]:
# Simulator + simple PI wheel-speed controller (cleaned & runnable)
import numpy as np
import matplotlib.pyplot as plt


# --- Helper: convert rpm/V to V/(rad/s) if needed (for kb)
def rpm_per_volt_to_kb(rpm_per_volt):
    # convert rpm/V -> rad/s per Volt
    rev_per_s_per_V = rpm_per_volt / 60.0
    rad_per_s_per_V = rev_per_s_per_V * 2 * np.pi
    # back-emf constant kb usually in V / (rad/s), so kb = 1 / (rad/s per V)
    return 1.0 / rad_per_s_per_V


params = {
    "dt": 0.01,
    "r": 0.05,  # wheel radius (m)
    "m": 2.0,  # robot mass (kg)
    "Jv": 0.125,  # rotational inertia (kg*m^2)
    "kt": 0.025,  # N*m / A
    "kb": rpm_per_volt_to_kb(460.0),  # V/(rad/s) approx, from 460 rpm/V spec
    "ra": 1.2,  # armature resistance (ohm)
    "ngb": 27.0,  # gearbox ratio (motor:wheel)
    "psi": np.deg2rad(np.array([90.0, 90.0, 90.0])),  # wheel traction angle psi_i
    "L": np.array([0.17, 0.17, 0.17]),  # distances Li
    "delta": np.deg2rad(
        np.array([0.0, 120.0, 240.0])
    ),  # wheel angular positions (robot fixed frame)
}

class Simulator:
    def __init__(self, params, omega0=np.zeros(3)):
        self.p = params.copy()
        self.p["alpha"] = self.p["delta"] - self.p["psi"]

        MC = np.zeros((3,3))
        for i in range(3):
            MC[i,0] = -np.cos(self.p["alpha"][i])
            MC[i,1] = -np.sin(self.p["alpha"][i])
            MC[i,2] = self.p["L"][i]*np.sin(self.p["psi"][i])
        self.MC = MC

        self.MM = np.diag([1/self.p["m"], 1/self.p["m"], 1/self.p["Jv"]])

        r, kt, kb, ra, ngb = (
            self.p["r"], self.p["kt"], self.p["kb"], self.p["ra"], self.p["ngb"]
        )

        self.MA = np.eye(3)*(kt*ngb/(ra*r))
        self.MB = np.eye(3)*(kt*kb*(ngb**2)/(ra*r))

        self.M1 = (1/r)*MC@self.MM@MC.T@self.MA
        self.M2 = (1/r)*MC@self.MM@MC.T@self.MB

        self.omega = omega0.copy()

    def step(self, v):
        omega_dot = self.M1 @ v - self.M2 @ self.omega
        self.omega += self.p["dt"] * omega_dot
        return self.omega.copy()

class GeneralMNRC:
    def __init__(self, f_func, g_func, Lambda_ref, zeta, tau, dt, n=3):
        self.f_func = f_func
        self.g_func = g_func
        self.Lambda = np.diag(Lambda_ref)  # For reference model (positive for stable poles)
        self.k = np.diag(2 * zeta / tau)   # Proportional gains
        self.k_I = np.diag(1 / (tau ** 2)) # Integral gains (for zeta=1)
        self.dt = dt
        self.y_m = np.zeros(n)
        self.e_I = np.zeros(n)

    def control(self, y, r):
        e = self.y_m - y
        dot_y_m = -self.Lambda @ (self.y_m - r)  # Reference model derivative
        f = self.f_func(y)
        g = self.g_func(y)
        inv_g = np.linalg.inv(g)
        u = inv_g @ (dot_y_m - f + self.k @ e + self.k_I @ self.e_I)
        
        # Update internal states (Euler integration)
        self.y_m += self.dt * dot_y_m
        self.e_I += self.dt * e
        
        return u
    
sim = Simulator(params, omega0=np.zeros(3))

# Define f and g for first level (wheel speeds)
f_func = lambda y: -sim.M2 @ y
g_func = lambda y: sim.M1

# Tuning (matching original Lambda=15, with zeta=1 for critical damping)
Lambda_ref = np.array([15.0, 15.0, 15.0])
zeta = np.array([1.0, 1.0, 1.0])
tau = 1.0 / Lambda_ref  # Time constants ~0.0667 s

ctrl = GeneralMNRC(f_func, g_func, Lambda_ref, zeta, tau, params["dt"])

T = 2.0
dt = params["dt"]
steps = int(T / dt)
t = np.linspace(0, T, steps)
omega_hist = np.zeros((3, steps))
v_hist = np.zeros((3, steps))

for k in range(steps):
    omega_r = np.array([15, 15, 15]) if t[k] > 0.2 else np.zeros(3)
    v = ctrl.control(y=sim.omega, r=omega_r)
    omega = sim.step(v)
    omega_hist[:, k] = omega
    v_hist[:, k] = v

for i in range(3):
    plt.figure(figsize=(6,3))
    plt.plot(t, omega_hist[i], label=rf"$\omega_{i+1}$")
    plt.axvline(0.2, ls="--", color="k")
    plt.xlabel("Time (s)")
    plt.ylabel("Wheel speed (rad/s)")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

for i in range(3):
    plt.figure(figsize=(6,3))
    plt.plot(t, v_hist[i], label=rf"$v_{i+1}$")
    plt.axvline(0.2, ls="--", color="k")
    plt.xlabel("Time (s)")
    plt.ylabel("Motor voltage (V)")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
# Simulator + simple PI wheel-speed controller (cleaned & runnable)
import numpy as np
import matplotlib.pyplot as plt


# --- Helper: convert rpm/V to V/(rad/s) if needed (for kb)
def rpm_per_volt_to_kb(rpm_per_volt):
    # convert rpm/V -> rad/s per Volt
    rev_per_s_per_V = rpm_per_volt / 60.0
    rad_per_s_per_V = rev_per_s_per_V * 2 * np.pi
    # back-emf constant kb usually in V / (rad/s), so kb = 1 / (rad/s per V)
    return 1.0 / rad_per_s_per_V


params = {
    "dt": 0.01,
    "r": 0.05,  # wheel radius (m)
    "m": 2.0,  # robot mass (kg)
    "Jv": 0.125,  # rotational inertia (kg*m^2)
    "kt": 0.025,  # N*m / A
    "kb": rpm_per_volt_to_kb(460.0),  # V/(rad/s) approx, from 460 rpm/V spec
    "ra": 1.2,  # armature resistance (ohm)
    "ngb": 27.0,  # gearbox ratio (motor:wheel)
    "psi": np.deg2rad(np.array([90.0, 90.0, 90.0])),  # wheel traction angle psi_i
    "L": np.array([0.17, 0.17, 0.17]),  # distances Li
    "delta": np.deg2rad(
        np.array([0.0, 120.0, 240.0])
    ),  # wheel angular positions (robot fixed frame)
}

class Simulator:
    def __init__(self, params, omega0=np.zeros(3), p0=np.zeros(3), vel0=np.zeros(3)):
        self.prms = params.copy()
        self.prms["alpha"] = self.prms["delta"] - self.prms["psi"]

        MC = np.zeros((3,3))
        for i in range(3):
            MC[i,0] = -np.cos(self.prms["alpha"][i])
            MC[i,1] = -np.sin(self.prms["alpha"][i])
            MC[i,2] = self.prms["L"][i]*np.sin(self.prms["psi"][i])
        self.MC = MC

        self.MM = np.diag([1/self.prms["m"], 1/self.prms["m"], 1/self.prms["Jv"]])

        r, kt, kb, ra, ngb = (
            self.prms["r"], self.prms["kt"], self.prms["kb"], self.prms["ra"], self.prms["ngb"]
        )

        self.MA = np.eye(3)*(kt*ngb/(ra*r))
        self.MB = np.eye(3)*(kt*kb*(ngb**2)/(ra*r))

        self.M1 = (1/r)*self.MC@self.MM@self.MC.T@self.MA
        self.M2 = (1/r)*self.MC@self.MM@self.MC.T@self.MB

        self.M3 = np.diag([15.0, 15.0, 15.0])  # from inner lambda_w
        self.M4 = r * np.linalg.inv(self.MC)

        self.omega = omega0.copy()
        self.p = p0.copy()
        self.vel = vel0.copy()

    def rotation(self, phi):
        c = np.cos(phi)
        s = np.sin(phi)
        return np.array([[c, s, 0.0], [-s, c, 0.0], [0.0, 0.0, 1.0]])

    def inner_step(self, v):
        omega_dot = self.M1 @ v - self.M2 @ self.omega
        self.omega += self.prms["dt"] * omega_dot

    def vehicle_step(self):
        phi = self.p[2]
        R = self.rotation(phi)
        self.vel = R.T @ np.linalg.inv(self.MC) @ (self.prms["r"] * self.omega)
        self.p += self.prms["dt"] * self.vel

class GeneralMNRC:
    def __init__(self, f_func, g_func, Lambda_ref, zeta, tau, dt, n=3):
        self.f_func = f_func
        self.g_func = g_func
        self.Lambda = np.diag(Lambda_ref)  # For reference model (positive for stable poles)
        self.k = np.diag(2 * zeta / tau)   # Proportional gains
        self.k_I = np.diag(1 / (tau ** 2)) # Integral gains (for zeta=1)
        self.dt = dt
        self.y_m = np.zeros(n)
        self.e_I = np.zeros(n)

    def control(self, y, r, **kwargs):
        dot_y_m = -self.Lambda @ (self.y_m - r)  # Reference model derivative
        f = self.f_func(y, **kwargs)
        g = self.g_func(y, **kwargs)
        inv_g = np.linalg.inv(g)
        e = self.y_m - y
        u = inv_g @ (dot_y_m - f + self.k @ e + self.k_I @ self.e_I)
        
        # Update internal states (Euler integration)
        self.y_m += self.dt * dot_y_m
        self.e_I += self.dt * e
        
        return u
    
sim = Simulator(params, omega0=np.zeros(3), p0=np.zeros(3), vel0=np.zeros(3))

# Define f and g for first level (wheel speeds)
f_func1 = lambda y, **kwargs: -sim.M2 @ y
g_func1 = lambda y, **kwargs: sim.M1

# Tuning for level 1 (matching original Lambda=15, with zeta=1 for critical damping)
Lambda_ref1 = np.array([15.0, 15.0, 15.0])
zeta1 = np.array([1.0, 1.0, 1.0])
tau1 = 1.0 / Lambda_ref1

ctrl1 = GeneralMNRC(f_func1, g_func1, Lambda_ref1, zeta1, tau1, params["dt"])

# Define f and g for second level (vehicle speeds)
f_func2 = lambda y, **kwargs: -sim.M3 @ y
g_func2 = lambda y, phi, **kwargs: sim.M3 @ sim.rotation(phi).T @ sim.M4

# Tuning for level 2 (slower, say Lambda=5)
Lambda_ref2 = np.array([5.0, 5.0, 5.0])
zeta2 = np.array([1.0, 1.0, 1.0])
tau2 = 1.0 / Lambda_ref2

ctrl2 = GeneralMNRC(f_func2, g_func2, Lambda_ref2, zeta2, tau2, params["dt"])

T = 2.0
dt = params["dt"]
steps = int(T / dt)
t = np.linspace(0, T, steps)
omega_hist = np.zeros((3, steps))
vel_hist = np.zeros((3, steps))
v_hist = np.zeros((3, steps))
r_omega_hist = np.zeros((3, steps))

for k in range(steps):
    r_v = np.array([0.5, 0.5, 2.0]) if t[k] > 0.2 else np.zeros(3)
    r_omega = ctrl2.control(y=sim.vel, r=r_v, phi=sim.p[2])
    v = ctrl1.control(y=sim.omega, r=r_omega)
    sim.inner_step(v)
    sim.vehicle_step()
    omega_hist[:, k] = sim.omega
    vel_hist[:, k] = sim.vel
    v_hist[:, k] = v
    r_omega_hist[:, k] = r_omega

labels_vel = [r"$\dot{x}_c^F$ (m/s)", r"$\dot{y}_c^F$ (m/s)", r"$\dot{\phi}_c^F$ (rad/s)"]
for i in range(3):
    plt.figure(figsize=(6,3))
    plt.plot(t, vel_hist[i], label=labels_vel[i])
    plt.axvline(0.2, ls="--", color="k")
    plt.xlabel("Time (s)")
    plt.ylabel("Vehicle speed")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

for i in range(3):
    plt.figure(figsize=(6,3))
    plt.plot(t, omega_hist[i], label=rf"$\omega_{i+1}$")
    plt.axvline(0.2, ls="--", color="k")
    plt.xlabel("Time (s)")
    plt.ylabel("Wheel speed (rad/s)")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

for i in range(3):
    plt.figure(figsize=(6,3))
    plt.plot(t, r_omega_hist[i], label=rf"$r\omega_{i+1}$")
    plt.axvline(0.2, ls="--", color="k")
    plt.xlabel("Time (s)")
    plt.ylabel("Wheel speed setpoint (rad/s)")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

for i in range(3):
    plt.figure(figsize=(6,3))
    plt.plot(t, v_hist[i], label=rf"$v_{i+1}$")
    plt.axvline(0.2, ls="--", color="k")
    plt.xlabel("Time (s)")
    plt.ylabel("Motor voltage (V)")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import deque


# --- Helper: convert rpm/V to V/(rad/s) if needed (for kb)
def rpm_per_volt_to_kb(rpm_per_volt):
    # convert rpm/V -> rad/s per Volt
    rev_per_s_per_V = rpm_per_volt / 60.0
    rad_per_s_per_V = rev_per_s_per_V * 2 * np.pi
    # back-emf constant kb usually in V / (rad/s), so kb = 1 / (rad/s per V)
    return 1.0 / rad_per_s_per_V


params = {
    "dt": 0.01,
    "theta": 0.1,  # dead time (s)
    "r": 0.05,  # wheel radius (m)
    "m": 2.0,  # robot mass (kg)
    "Jv": 0.125,  # rotational inertia (kg*m^2)
    "kt": 0.025,  # N*m / A
    "kb": rpm_per_volt_to_kb(460.0),  # V/(rad/s) approx, from 460 rpm/V spec
    "ra": 1.2,  # armature resistance (ohm)
    "ngb": 27.0,  # gearbox ratio (motor:wheel)
    "psi": np.deg2rad(np.array([90.0, 90.0, 90.0])),  # wheel traction angle psi_i
    "L": np.array([0.17, 0.17, 0.17]),  # distances Li
    "delta": np.deg2rad(
        np.array([0.0, 120.0, 240.0])
    ),  # wheel angular positions (robot fixed frame)
}

class Simulator:
    def __init__(self, params, omega0=np.zeros(3), p0=np.zeros(3), vel0=np.zeros(3)):
        self.prms = params.copy()
        self.prms["alpha"] = self.prms["delta"] - self.prms["psi"]

        MC = np.zeros((3,3))
        for i in range(3):
            MC[i,0] = -np.cos(self.prms["alpha"][i])
            MC[i,1] = -np.sin(self.prms["alpha"][i])
            MC[i,2] = self.prms["L"][i]*np.sin(self.prms["psi"][i])
        self.MC = MC

        self.MM = np.diag([1/self.prms["m"], 1/self.prms["m"], 1/self.prms["Jv"]])

        r, kt, kb, ra, ngb = (
            self.prms["r"], self.prms["kt"], self.prms["kb"], self.prms["ra"], self.prms["ngb"]
        )

        self.MA = np.eye(3)*(kt*ngb/(ra*r))
        self.MB = np.eye(3)*(kt*kb*(ngb**2)/(ra*r))

        self.M1 = (1/r)*MC@self.MM@MC.T@self.MA
        self.M2 = (1/r)*MC@self.MM@MC.T@self.MB

        self.M3 = np.diag([15.0, 15.0, 15.0])  # from inner lambda_w
        self.M4 = r * np.linalg.inv(self.MC)

        self.omega = omega0.copy()
        self.p = p0.copy()
        self.vel = vel0.copy()

        self.delay_steps = int(self.prms["theta"] / self.prms["dt"])
        self.vel_history = deque([np.zeros(3) for _ in range(self.delay_steps + 1)], maxlen=self.delay_steps + 1)
        self.p_history = deque([np.zeros(3) for _ in range(self.delay_steps + 1)], maxlen=self.delay_steps + 1)

    def rotation(self, phi):
        c = np.cos(phi)
        s = np.sin(phi)
        return np.array([[c, s, 0.0], [-s, c, 0.0], [0.0, 0.0, 1.0]])

    def inner_step(self, v):
        omega_dot = self.M1 @ v - self.M2 @ self.omega
        self.omega += self.prms["dt"] * omega_dot

    def vehicle_step(self):
        phi = self.p[2]
        R = self.rotation(phi)
        self.vel = R.T @ np.linalg.inv(self.MC) @ (self.prms["r"] * self.omega)
        self.p += self.prms["dt"] * self.vel

        self.vel_history.append(self.vel.copy())
        self.p_history.append(self.p.copy())

class GeneralMNRC:
    def __init__(self, f_func, g_func, Lambda_ref, zeta, tau, dt, n=3):
        self.f_func = f_func
        self.g_func = g_func
        self.Lambda = np.diag(Lambda_ref)  # For reference model (positive for stable poles)
        self.k = np.diag(2 * zeta / tau)   # Proportional gains
        self.k_I = np.diag(1 / (tau ** 2)) # Integral gains (for zeta=1)
        self.dt = dt
        self.y_m = np.zeros(n)
        self.e_I = np.zeros(n)

    def control(self, y, r, **kwargs):
        dot_y_m = -self.Lambda @ (self.y_m - r)  # Reference model derivative
        f = self.f_func(y, **kwargs)
        g = self.g_func(y, **kwargs)
        inv_g = np.linalg.inv(g)
        e = self.y_m - y
        u = inv_g @ (dot_y_m - f + self.k @ e + self.k_I @ self.e_I)
        
        # Update internal states (Euler integration)
        self.y_m += self.dt * dot_y_m
        self.e_I += self.dt * e
        
        return u
    
sim = Simulator(params, omega0=np.zeros(3), p0=np.zeros(3), vel0=np.zeros(3))

# Define f and g for first level (wheel speeds)
f_func1 = lambda y, **kwargs: -sim.M2 @ y
g_func1 = lambda y, **kwargs: sim.M1

# Tuning for level 1 (matching original Lambda=15, with zeta=1 for critical damping)
Lambda_ref1 = np.array([15.0, 15.0, 15.0])
zeta1 = np.array([1.0, 1.0, 1.0])
tau1 = 1.0 / Lambda_ref1

ctrl1 = GeneralMNRC(f_func1, g_func1, Lambda_ref1, zeta1, tau1, params["dt"])

# Define f and g for second level (vehicle speeds) with corrected signs
f_func2 = lambda y, **kwargs: -sim.M3 @ y
g_func2 = lambda y, phi, **kwargs: sim.M3 @ sim.rotation(phi).T @ sim.M4

# Tuning for level 2 (slower, say Lambda=5)
Lambda_ref2 = np.array([5.0, 5.0, 5.0])
zeta2 = np.array([1.0, 1.0, 1.0])
tau2 = 1.0 / Lambda_ref2

ctrl2 = GeneralMNRC(f_func2, g_func2, Lambda_ref2, zeta2, tau2, params["dt"])

# Define f and g for third level (vehicle position)
f_func3 = lambda y, **kwargs: np.zeros(3)
g_func3 = lambda y, **kwargs: np.eye(3)

# Tuning for level 3 (even slower, say Lambda=1)
Lambda_ref3 = np.array([1.0, 1.0, 1.0])
zeta3 = np.array([1.0, 1.0, 1.0])
tau3 = 1.0 / Lambda_ref3

ctrl3 = GeneralMNRC(f_func3, g_func3, Lambda_ref3, zeta3, tau3, params["dt"])

T = 2.0
dt = params["dt"]
steps = int(T / dt)
t = np.linspace(0, T, steps)
omega_hist = np.zeros((3, steps))
vel_hist = np.zeros((3, steps))
p_hist = np.zeros((3, steps))
v_hist = np.zeros((3, steps))
r_omega_hist = np.zeros((3, steps))
r_v_hist = np.zeros((3, steps))

for k in range(steps):
    r_p = np.array([0.5, 0.5, np.pi]) if t[k] > 0.2 else np.zeros(3)

    p_m = sim.p_history[0]

    r_v = ctrl3.control(y=p_m, r=r_p)

    phi_m = p_m[2]

    vel_m = sim.vel_history[0]

    r_omega = ctrl2.control(y=vel_m, r=r_v, phi=phi_m)

    v = ctrl1.control(y=sim.omega, r=r_omega)

    sim.inner_step(v)
    sim.vehicle_step()

    omega_hist[:, k] = sim.omega
    vel_hist[:, k] = sim.vel
    p_hist[:, k] = sim.p
    v_hist[:, k] = v
    r_omega_hist[:, k] = r_omega
    r_v_hist[:, k] = r_v

labels_p = [r"$x_c^F$ (m)", r"$y_c^F$ (m)", r"$\phi_c^F$ (rad)"]
for i in range(3):
    plt.figure(figsize=(6,3))
    plt.plot(t, p_hist[i], label=labels_p[i])
    plt.axvline(0.2, ls="--", color="k")
    plt.xlabel("Time (s)")
    plt.ylabel("Vehicle position")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

labels_vel = [r"$\dot{x}_c^F$ (m/s)", r"$\dot{y}_c^F$ (m/s)", r"$\dot{\phi}_c^F$ (rad/s)"]
for i in range(3):
    plt.figure(figsize=(6,3))
    plt.plot(t, vel_hist[i], label=labels_vel[i])
    plt.axvline(0.2, ls="--", color="k")
    plt.xlabel("Time (s)")
    plt.ylabel("Vehicle speed")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

for i in range(3):
    plt.figure(figsize=(6,3))
    plt.plot(t, r_v_hist[i], label=rf"$r_{{v {i+1}}}$")
    plt.axvline(0.2, ls="--", color="k")
    plt.xlabel("Time (s)")
    plt.ylabel("Vehicle speed setpoint")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

for i in range(3):
    plt.figure(figsize=(6,3))
    plt.plot(t, omega_hist[i], label=rf"$\omega_{i+1}$")
    plt.axvline(0.2, ls="--", color="k")
    plt.xlabel("Time (s)")
    plt.ylabel("Wheel speed (rad/s)")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

for i in range(3):
    plt.figure(figsize=(6,3))
    plt.plot(t, r_omega_hist[i], label=rf"$r\omega_{i+1}$")
    plt.axvline(0.2, ls="--", color="k")
    plt.xlabel("Time (s)")
    plt.ylabel("Wheel speed setpoint (rad/s)")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

for i in range(3):
    plt.figure(figsize=(6,3))
    plt.plot(t, v_hist[i], label=rf"$v_{i+1}$")
    plt.axvline(0.2, ls="--", color="k")
    plt.xlabel("Time (s)")
    plt.ylabel("Motor voltage (V)")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
# Simulator + simple PI wheel-speed controller (cleaned & runnable)
import numpy as np
import matplotlib.pyplot as plt
from collections import deque


# --- Helper: convert rpm/V to V/(rad/s) if needed (for kb)
def rpm_per_volt_to_kb(rpm_per_volt):
    rev_per_s_per_V = rpm_per_volt / 60.0
    rad_per_s_per_V = rev_per_s_per_V * 2 * np.pi
    return 1.0 / rad_per_s_per_V


params = {
    "dt": 0.01,
    "theta": 0.1,  # dead time (s)
    "r": 0.05,  # wheel radius (m)
    "m": 2.0,  # robot mass (kg)
    "Jv": 0.125,  # rotational inertia (kg*m^2)
    "kt": 0.025,  # N*m / A
    "kb": rpm_per_volt_to_kb(460.0),  # V/(rad/s) approx, from 460 rpm/V spec
    "ra": 1.2,  # armature resistance (ohm)
    "ngb": 27.0,  # gearbox ratio (motor:wheel)
    "psi": np.deg2rad(np.array([90.0, 90.0, 90.0])),  # wheel traction angle psi_i
    "L": np.array([0.17, 0.17, 0.17]),  # distances Li
    "delta": np.deg2rad(
        np.array([0.0, 120.0, 240.0])
    ),  # wheel angular positions (robot fixed frame)
    "noise_p": 0.01,
    "noise_v": 0.005,
}

class Simulator:
    def __init__(self, params, omega0=np.zeros(3), p0=np.zeros(3), vel0=np.zeros(3)):
        self.prms = params.copy()
        self.prms["alpha"] = self.prms["delta"] - self.prms["psi"]

        MC = np.zeros((3,3))
        for i in range(3):
            MC[i,0] = -np.cos(self.prms["alpha"][i])
            MC[i,1] = -np.sin(self.prms["alpha"][i])
            MC[i,2] = self.prms["L"][i]*np.sin(self.prms["psi"][i])
        self.MC = MC

        self.MM = np.diag([1/self.prms["m"], 1/self.prms["m"], 1/self.prms["Jv"]])

        r, kt, kb, ra, ngb = (
            self.prms["r"], self.prms["kt"], self.prms["kb"], self.prms["ra"], self.prms["ngb"]
        )

        self.MA = np.eye(3)*(kt*ngb/(ra*r))
        self.MB = np.eye(3)*(kt*kb*(ngb**2)/(ra*r))

        self.M1 = (1/r)*MC@self.MM@MC.T@self.MA
        self.M2 = (1/r)*MC@self.MM@MC.T@self.MB

        self.M3 = np.diag([15.0, 15.0, 15.0])  # lambda_w
        self.M4 = r * np.linalg.inv(self.MC)

        self.omega = omega0.copy()
        self.p = p0.copy()
        self.vel = vel0.copy()

        self.delay_steps = int(self.prms["theta"] / self.prms["dt"])
        self.vel_history = deque([np.zeros(3) for _ in range(self.delay_steps + 1)], maxlen=self.delay_steps + 1)
        self.p_history = deque([np.zeros(3) for _ in range(self.delay_steps + 1)], maxlen=self.delay_steps + 1)

    def rotation(self, phi):
        c = np.cos(phi)
        s = np.sin(phi)
        return np.array([[c, s, 0.0], [-s, c, 0.0], [0.0, 0.0, 1.0]])

    def inner_step(self, v):
        omega_dot = self.M1 @ v - self.M2 @ self.omega
        self.omega += self.prms["dt"] * omega_dot

    def vehicle_step(self):
        phi = self.p[2]
        R = self.rotation(phi)
        self.vel = R.T @ np.linalg.inv(self.MC) @ (self.prms["r"] * self.omega)
        self.p += self.prms["dt"] * self.vel

        self.vel_history.append(self.vel.copy())
        self.p_history.append(self.p.copy())

class GeneralMNRC:
    def __init__(self, f_func, g_func, Lambda_ref, zeta, tau, dt, n=3):
        self.f_func = f_func
        self.g_func = g_func
        self.Lambda = np.diag(Lambda_ref)
        self.k = np.diag(2 * zeta / tau)
        self.k_I = np.diag(1 / (tau ** 2))
        self.dt = dt
        self.y_m = np.zeros(n)
        self.e_I = np.zeros(n)

    def control(self, y, r, **kwargs):
        dot_y_m = -self.Lambda @ (self.y_m - r)
        f = self.f_func(y, **kwargs)
        g = self.g_func(y, **kwargs)
        inv_g = np.linalg.inv(g)
        e = self.y_m - y
        u = inv_g @ (dot_y_m - f + self.k @ e + self.k_I @ self.e_I)
        
        self.y_m += self.dt * dot_y_m
        self.e_I += self.dt * e
        
        return u

class SmithPredictor:
    def __init__(self, lambda_v, dt, delay_steps):
        self.lambda_v = lambda_v
        self.dt = dt
        self.model_vel = np.zeros(3)
        self.delay_buffer = deque([np.zeros(3) for _ in range(delay_steps + 1)], maxlen=delay_steps + 1)

    def get_predicted(self, vel_m):
        model_delayed = self.delay_buffer[0]
        predicted = self.model_vel + (vel_m - model_delayed)
        return predicted

    def update(self, r_v):
        model_vel_dot = -self.lambda_v * (self.model_vel - r_v)
        self.model_vel += self.dt * model_vel_dot
        self.delay_buffer.append(self.model_vel.copy())

class ModifiedSmithPredictor:
    def __init__(self, lambda_v, P1, P2, dt, delay_steps):
        self.lambda_v = lambda_v
        self.P1 = P1
        self.P2 = P2
        self.dt = dt
        self.model_vel = np.zeros(3)
        self.model_p = np.zeros(3)
        self.delay_buffer = deque([np.zeros(3) for _ in range(delay_steps + 1)], maxlen=delay_steps + 1)

    def control(self, r_p, p_m):
        model_delayed = self.delay_buffer[0]
        predicted_p = self.model_p + self.P2 * (p_m - model_delayed)
        e = r_p - predicted_p
        r_v = self.P1 * e
        return r_v

    def update(self, r_v):
        model_vel_dot = -self.lambda_v * (self.model_vel - r_v)
        self.model_vel += self.dt * model_vel_dot
        self.model_p += self.dt * self.model_vel
        self.delay_buffer.append(self.model_p.copy())

sim = Simulator(params)

f_func1 = lambda y, **kwargs: -sim.M2 @ y
g_func1 = lambda y, **kwargs: sim.M1

Lambda_ref1 = np.array([15.0, 15.0, 15.0])
zeta1 = np.array([1.0, 1.0, 1.0])
tau1 = 1.0 / Lambda_ref1

ctrl1 = GeneralMNRC(f_func1, g_func1, Lambda_ref1, zeta1, tau1, params["dt"])

f_func2 = lambda y, **kwargs: -sim.M3 @ y
g_func2 = lambda y, phi, **kwargs: sim.M3 @ sim.rotation(phi).T @ sim.M4

Lambda_ref2 = np.array([5.0, 5.0, 5.0])
zeta2 = np.array([1.0, 1.0, 1.0])
tau2 = 1.0 / Lambda_ref2

ctrl2 = GeneralMNRC(f_func2, g_func2, Lambda_ref2, zeta2, tau2, params["dt"])

smith2 = SmithPredictor(Lambda_ref2, params["dt"], sim.delay_steps)

P1 = np.array([0.5, 0.5, 0.5])
P2 = np.array([2.0, 2.0, 2.0])
mod_smith3 = ModifiedSmithPredictor(Lambda_ref2, P1, P2, params["dt"], sim.delay_steps)

T = 12.0
dt = params["dt"]
steps = int(T / dt)
t = np.linspace(0, T, steps)
omega_hist = np.zeros((3, steps))
vel_hist = np.zeros((3, steps))
p_hist = np.zeros((3, steps))
v_hist = np.zeros((3, steps))
r_omega_hist = np.zeros((3, steps))
r_v_hist = np.zeros((3, steps))
r_p_hist = np.zeros((3, steps))

w = 2 * np.pi / 8.0  # approximate period 8 s
k_phi = 10.0 / 12.0

for k in range(steps):
    r_p = np.array([0.3 * np.sin(w * t[k]), 0.3 * np.cos(w * t[k]), - k_phi * t[k]])
    r_p_hist[:, k] = r_p

    p_m = sim.p_history[0] + np.random.normal(0, params["noise_p"], 3)
    vel_m = sim.vel_history[0] + np.random.normal(0, params["noise_v"], 3)
    phi_m = p_m[2]

    r_v = mod_smith3.control(r_p, p_m)
    mod_smith3.update(r_v)
    r_v_hist[:, k] = r_v

    predicted_vel = smith2.get_predicted(vel_m)

    r_omega = ctrl2.control(predicted_vel, r_v, phi=phi_m)
    r_omega_hist[:, k] = r_omega

    smith2.update(r_v)

    v = ctrl1.control(sim.omega, r_omega)
    v_hist[:, k] = v

    sim.inner_step(v)
    sim.vehicle_step()

    omega_hist[:, k] = sim.omega
    vel_hist[:, k] = sim.vel
    p_hist[:, k] = sim.p

fig, axs = plt.subplots(3, 1, figsize=(8, 6), sharex=True)
labels_p = [r"$x_c^F$ (m)", r"$y_c^F$ (m)", r"$\phi_c^F$ (rad)"]
for i in range(3):
    axs[i].plot(t, p_hist[i], label='Measured')
    axs[i].plot(t, r_p_hist[i], '--', label='Setpoint')
    axs[i].set_ylabel(labels_p[i])
    axs[i].grid(True)
    axs[i].legend()
axs[2].set_xlabel("Time (s)")
fig.suptitle("Vehicle Position")
plt.tight_layout()

fig, axs = plt.subplots(3, 1, figsize=(8, 6), sharex=True)
labels_vel = [r"$\dot{x}_c^F$ (m/s)", r"$\dot{y}_c^F$ (m/s)", r"$\dot{\phi}_c^F$ (rad/s)"]
for i in range(3):
    axs[i].plot(t, vel_hist[i], label='Velocity')
    axs[i].plot(t, r_v_hist[i], '--', label='Setpoint')
    axs[i].set_ylabel(labels_vel[i])
    axs[i].grid(True)
    axs[i].legend()
axs[2].set_xlabel("Time (s)")
fig.suptitle("Vehicle Velocity")
plt.tight_layout()

fig, axs = plt.subplots(3, 1, figsize=(8, 6), sharex=True)
for i in range(3):
    axs[i].plot(t, omega_hist[i], label=rf"$\omega_{i+1}$")
    axs[i].plot(t, r_omega_hist[i], '--', label='Setpoint')
    axs[i].set_ylabel("Wheel speed (rad/s)")
    axs[i].grid(True)
    axs[i].legend()
axs[2].set_xlabel("Time (s)")
fig.suptitle("Wheel Speeds")
plt.tight_layout()

fig, axs = plt.subplots(3, 1, figsize=(8, 6), sharex=True)
for i in range(3):
    axs[i].plot(t, v_hist[i], label=rf"$v_{i+1}$")
    axs[i].set_ylabel("Motor voltage (V)")
    axs[i].grid(True)
    axs[i].legend()
axs[2].set_xlabel("Time (s)")
fig.suptitle("Motor Voltages")
plt.tight_layout()

In [ ]:
# Simulator + simple PI wheel-speed controller (cleaned & runnable)
import numpy as np
import matplotlib.pyplot as plt
from collections import deque


# --- Helper: convert rpm/V to V/(rad/s) if needed (for kb)
def rpm_per_volt_to_kb(rpm_per_volt):
    rev_per_s_per_V = rpm_per_volt / 60.0
    rad_per_s_per_V = rev_per_s_per_V * 2 * np.pi
    return 1.0 / rad_per_s_per_V


params = {
    "dt": 0.01,
    "theta": 0.1,  # dead time (s)
    "r": 0.05,  # wheel radius (m)
    "m": 2.0,  # robot mass (kg)
    "Jv": 0.125,  # rotational inertia (kg*m^2)
    "kt": 0.025,  # N*m / A
    "kb": rpm_per_volt_to_kb(460.0),  # V/(rad/s) approx, from 460 rpm/V spec
    "ra": 1.2,  # armature resistance (ohm)
    "ngb": 27.0,  # gearbox ratio (motor:wheel)
    "psi": np.deg2rad(np.array([90.0, 90.0, 90.0])),  # wheel traction angle psi_i
    "L": np.array([0.17, 0.17, 0.17]),  # distances Li
    "delta": np.deg2rad(
        np.array([0.0, 120.0, 240.0])
    ),  # wheel angular positions (robot fixed frame)
    "noise_p": 0.01,
    "noise_v": 0.005,
}

class Simulator:
    def __init__(self, params, omega0=np.zeros(3), p0=np.zeros(3), vel0=np.zeros(3)):
        self.prms = params.copy()
        self.prms["alpha"] = self.prms["delta"] - self.prms["psi"]

        MC = np.zeros((3,3))
        for i in range(3):
            MC[i,0] = -np.cos(self.prms["alpha"][i])
            MC[i,1] = -np.sin(self.prms["alpha"][i])
            MC[i,2] = self.prms["L"][i]*np.sin(self.prms["psi"][i])
        self.MC = MC

        self.MM = np.diag([1/self.prms["m"], 1/self.prms["m"], 1/self.prms["Jv"]])

        r, kt, kb, ra, ngb = (
            self.prms["r"], self.prms["kt"], self.prms["kb"], self.prms["ra"], self.prms["ngb"]
        )

        self.MA = np.eye(3)*(kt*ngb/(ra*r))
        self.MB = np.eye(3)*(kt*kb*(ngb**2)/(ra*r))

        self.M1 = (1/r)*MC@self.MM@MC.T@self.MA
        self.M2 = (1/r)*MC@self.MM@MC.T@self.MB

        self.M3 = np.diag([15.0, 15.0, 15.0])  # lambda_w
        self.M4 = r * np.linalg.inv(self.MC)

        self.omega = omega0.copy()
        self.p = p0.copy()
        self.vel = vel0.copy()

        self.delay_steps = int(self.prms["theta"] / self.prms["dt"])
        self.vel_history = deque([np.zeros(3) for _ in range(self.delay_steps + 1)], maxlen=self.delay_steps + 1)
        self.p_history = deque([np.zeros(3) for _ in range(self.delay_steps + 1)], maxlen=self.delay_steps + 1)

    def rotation(self, phi):
        c = np.cos(phi)
        s = np.sin(phi)
        return np.array([[c, s, 0.0], [-s, c, 0.0], [0.0, 0.0, 1.0]])

    def inner_step(self, v):
        omega_dot = self.M1 @ v - self.M2 @ self.omega
        self.omega += self.prms["dt"] * omega_dot

    def vehicle_step(self):
        phi = self.p[2]
        R = self.rotation(phi)
        self.vel = R.T @ np.linalg.inv(self.MC) @ (self.prms["r"] * self.omega)
        self.p += self.prms["dt"] * self.vel

        self.vel_history.append(self.vel.copy() + np.random.normal(0, self.prms["noise_v"], 3))
        self.p_history.append(self.p.copy() + np.random.normal(0, self.prms["noise_p"], 3))

class GeneralMNRC:
    def __init__(self, f_func, g_func, Lambda_ref, zeta, tau, dt, n=3):
        self.f_func = f_func
        self.g_func = g_func
        self.Lambda = np.diag(Lambda_ref)
        self.k = np.diag(2 * zeta / tau)
        self.k_I = np.diag(1 / (tau ** 2))
        self.dt = dt
        self.y_m = np.zeros(n)
        self.e_I = np.zeros(n)

    def control(self, y, r, **kwargs):
        dot_y_m = -self.Lambda @ (self.y_m - r)
        f = self.f_func(y, **kwargs)
        g = self.g_func(y, **kwargs)
        inv_g = np.linalg.inv(g)
        e = self.y_m - y
        u = inv_g @ (dot_y_m - f + self.k @ e + self.k_I @ self.e_I)
        
        self.y_m += self.dt * dot_y_m
        self.e_I += self.dt * e
        
        return u

class SmithPredictor:
    def __init__(self, lambda_v, dt, delay_steps):
        self.lambda_v = lambda_v
        self.dt = dt
        self.model_vel = np.zeros(3)
        self.delay_buffer = deque([np.zeros(3) for _ in range(delay_steps + 1)], maxlen=delay_steps + 1)

    def get_predicted(self, vel_m):
        model_delayed = self.delay_buffer[0]
        predicted = self.model_vel + (vel_m - model_delayed)
        return predicted

    def update(self, r_v):
        model_vel_dot = -self.lambda_v * (self.model_vel - r_v)
        self.model_vel += self.dt * model_vel_dot
        self.delay_buffer.append(self.model_vel.copy())

class ModifiedSmithPredictor:
    def __init__(self, lambda_v, P1, P2, dt, delay_steps):
        self.lambda_v = lambda_v
        self.P1 = P1
        self.P2 = P2
        self.dt = dt
        self.model_vel = np.zeros(3)
        self.model_p = np.zeros(3)
        self.delay_buffer = deque([np.zeros(3) for _ in range(delay_steps + 1)], maxlen=delay_steps + 1)

    def control(self, r_p, p_m):
        model_delayed = self.delay_buffer[0]
        predicted_p = self.model_p + self.P2 * (p_m - model_delayed)
        e = r_p - predicted_p
        r_v = self.P1 * e
        return r_v

    def update(self, r_v):
        model_vel_dot = -self.lambda_v * (self.model_vel - r_v)
        self.model_vel += self.dt * model_vel_dot
        self.model_p += self.dt * self.model_vel
        self.delay_buffer.append(self.model_p.copy())

sim = Simulator(params)

f_func1 = lambda y, **kwargs: -sim.M2 @ y
g_func1 = lambda y, **kwargs: sim.M1

Lambda_ref1 = np.array([15.0, 15.0, 15.0])
zeta1 = np.array([1.0, 1.0, 1.0])
tau1 = 1.0 / Lambda_ref1

ctrl1 = GeneralMNRC(f_func1, g_func1, Lambda_ref1, zeta1, tau1, params["dt"])

f_func2 = lambda y, **kwargs: -sim.M3 @ y
g_func2 = lambda y, phi, **kwargs: sim.M3 @ sim.rotation(phi).T @ sim.M4

Lambda_ref2 = np.array([5.0, 5.0, 5.0])
zeta2 = np.array([1.0, 1.0, 1.0])
tau2 = 1.0 / Lambda_ref2

ctrl2 = GeneralMNRC(f_func2, g_func2, Lambda_ref2, zeta2, tau2, params["dt"])

smith2 = SmithPredictor(Lambda_ref2, params["dt"], sim.delay_steps)

P1 = np.array([0.5, 0.5, 0.5])
P2 = np.array([2.0, 2.0, 2.0])
mod_smith3 = ModifiedSmithPredictor(Lambda_ref2, P1, P2, params["dt"], sim.delay_steps)

T = 12.0
dt = params["dt"]
steps = int(T / dt)
t = np.linspace(0, T, steps)
omega_hist = np.zeros((3, steps))
vel_hist = np.zeros((3, steps))
p_hist = np.zeros((3, steps))
v_hist = np.zeros((3, steps))
r_omega_hist = np.zeros((3, steps))
r_v_hist = np.zeros((3, steps))
r_p_hist = np.zeros((3, steps))

w = 2 * np.pi / 8.0  # approximate period 8 s
k_phi = 10.0 / 12.0

for k in range(steps):
    r_p = np.array([0.3 * np.sin(w * t[k]), 0.3 * np.cos(w * t[k]), - k_phi * t[k]])
    r_p_hist[:, k] = r_p

    p_m = sim.p_history[0]
    vel_m = sim.vel_history[0]
    phi_m = p_m[2]

    r_v = mod_smith3.control(r_p, p_m)
    mod_smith3.update(r_v)
    r_v_hist[:, k] = r_v

    predicted_vel = smith2.get_predicted(vel_m)

    r_omega = ctrl2.control(predicted_vel, r_v, phi=phi_m)
    r_omega_hist[:, k] = r_omega

    smith2.update(r_v)

    v = ctrl1.control(sim.omega, r_omega)
    v_hist[:, k] = v

    sim.inner_step(v)
    sim.vehicle_step()

    omega_hist[:, k] = sim.omega
    vel_hist[:, k] = sim.vel
    p_hist[:, k] = sim.p

fig, axs = plt.subplots(3, 1, figsize=(8, 6), sharex=True)
labels_p = [r"$x_c^F$ (m)", r"$y_c^F$ (m)", r"$\phi_c^F$ (rad)"]
for i in range(3):
    axs[i].plot(t, p_hist[i], label='Measured')
    axs[i].plot(t, r_p_hist[i], '--', label='Setpoint')
    axs[i].set_ylabel(labels_p[i])
    axs[i].grid(True)
    axs[i].legend()
axs[2].set_xlabel("Time (s)")
fig.suptitle("Vehicle Position")
plt.tight_layout()

fig, axs = plt.subplots(3, 1, figsize=(8, 6), sharex=True)
labels_vel = [r"$\dot{x}_c^F$ (m/s)", r"$\dot{y}_c^F$ (m/s)", r"$\dot{\phi}_c^F$ (rad/s)"]
for i in range(3):
    axs[i].plot(t, vel_hist[i], label='Velocity')
    axs[i].plot(t, r_v_hist[i], '--', label='Setpoint')
    axs[i].set_ylabel(labels_vel[i])
    axs[i].grid(True)
    axs[i].legend()
axs[2].set_xlabel("Time (s)")
fig.suptitle("Vehicle Velocity")
plt.tight_layout()

fig, axs = plt.subplots(3, 1, figsize=(8, 6), sharex=True)
for i in range(3):
    axs[i].plot(t, omega_hist[i], label=rf"$\omega_{i+1}$")
    axs[i].plot(t, r_omega_hist[i], '--', label='Setpoint')
    axs[i].set_ylabel("Wheel speed (rad/s)")
    axs[i].grid(True)
    axs[i].legend()
axs[2].set_xlabel("Time (s)")
fig.suptitle("Wheel Speeds")
plt.tight_layout()

fig, axs = plt.subplots(3, 1, figsize=(8, 6), sharex=True)
for i in range(3):
    axs[i].plot(t, v_hist[i], label=rf"$v_{i+1}$")
    axs[i].set_ylabel("Motor voltage (V)")
    axs[i].grid(True)
    axs[i].legend()
axs[2].set_xlabel("Time (s)")
fig.suptitle("Motor Voltages")
plt.tight_layout()